In [10]:
import requests
years= list(range(1991,2024))
url_start='https://www.basketball-reference.com/awards/awards_{}.html'

In [5]:
for year in years:
    url=url_start.format(year)
    data=requests.get(url)
    
    with open("mvp/{}.html".format(year), "w+",encoding='utf-8') as f:
        f.write(data.text)

In [14]:
from bs4 import BeautifulSoup

In [10]:
with open("mvp/1991.html", encoding='utf-8') as f:
    page=f.read()

In [15]:
soup=BeautifulSoup(page,'html.parser')

In [14]:
soup.find('tr', class_="over_header").decompose()

In [18]:
mvp_table=soup.find(id='mvp')

In [20]:
import pandas as pd
from io import StringIO

In [24]:
mvp_1991=pd.read_html(StringIO(str(mvp_table)))[0]

In [32]:
dfs=[]
for year in years:
    with open("mvp/{}.html".format(year),encoding="utf-8") as f:
        page=f.read()
    soup=BeautifulSoup(page,'html.parser')
    soup.find('tr', class_="over_header").decompose()
    mvp_table=soup.find(id='mvp')
    mvp=pd.read_html(StringIO(str(mvp_table)))[0]
    mvp["Year"]=year
    dfs.append(mvp)


In [33]:
mvps=pd.concat(dfs)

In [38]:
mvps.to_csv("mvps.csv")

In [7]:
player_stats_url="https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
url=player_stats_url.format(1991)
data=requests.get(url)
with open("player/1991.html","w+",encoding='utf-8') as f:
    f.write(data.text)

In [8]:
from selenium import webdriver

In [9]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Assuming you have defined player_stats_url and year earlier
driver = webdriver.Chrome()
year=1991
url = player_stats_url.format(year)
driver.get(url)

# Use WebDriverWait to wait for the presence of an element before scrolling
wait = WebDriverWait(driver, 10)  # Maximum wait time of 10 seconds
element_to_wait_for = wait.until(EC.presence_of_element_located((By.ID, "some_element_id")))

# Scroll to the bottom of the page
#driver.execute_script("arguments[0].scrollIntoView();", element_to_wait_for)

# If the above doesn't work, try using window.scrollTo directly
driver.execute_script("window.scrollTo(1, 10000)")

# Give some time for the scrolling to take effect
time.sleep(2)

# Continue with the rest of your code
html = driver.page_source
with open("player/{}.html".format(year),"w+", encoding='utf-8') as f:
    f.write(html)

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6DBE24D02+56194]
	(No symbol) [0x00007FF6DBD904B2]
	(No symbol) [0x00007FF6DBC376AA]
	(No symbol) [0x00007FF6DBC816D0]
	(No symbol) [0x00007FF6DBC817EC]
	(No symbol) [0x00007FF6DBCC4D77]
	(No symbol) [0x00007FF6DBCA5EBF]
	(No symbol) [0x00007FF6DBCC2786]
	(No symbol) [0x00007FF6DBCA5C23]
	(No symbol) [0x00007FF6DBC74A45]
	(No symbol) [0x00007FF6DBC75AD4]
	GetHandleVerifier [0x00007FF6DC19D5BB+3695675]
	GetHandleVerifier [0x00007FF6DC1F6197+4059159]
	GetHandleVerifier [0x00007FF6DC1EDF63+4025827]
	GetHandleVerifier [0x00007FF6DBEBF029+687785]
	(No symbol) [0x00007FF6DBD9B508]
	(No symbol) [0x00007FF6DBD97564]
	(No symbol) [0x00007FF6DBD976E9]
	(No symbol) [0x00007FF6DBD88094]
	BaseThreadInitThunk [0x00007FFB9CC7257D+29]
	RtlUserThreadStart [0x00007FFB9E36AA58+40]


In [2]:
team_stats_url="https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [11]:
import requests
for year in years:
    url=team_stats_url.format(year)
    data=requests.get(url)
    with open("team/{}.html".format(year), "w+",encoding="utf-8") as f:
        f.write(data.text)

In [30]:
dfs=[]
for year in years:
    with open("team/{}.html".format(year),encoding='utf-8') as f:
        page=f.read()
    
    soup=BeautifulSoup(page,"html.parser")
    soup.find("tr", class_="thead").decompose()
    team_table=soup.find(id="divs_standings_E")
    team=pd.read_html(StringIO(str(team_table)))[0]
    team["Year"]=year
    team["Team"]=team["Eastern Conference"]
    del team['Eastern Conference']
    dfs.append(team)
    
    soup=BeautifulSoup(page,"html.parser")
    soup.find("tr", class_="thead").decompose()
    team_table=soup.find(id="divs_standings_W")
    team=pd.read_html(StringIO(str(team_table)))[0]
    team["Year"]=year
    team["Team"]=team["Western Conference"]
    del team['Western Conference']
    dfs.append(team)

In [31]:
teams=pd.concat(dfs)

In [32]:
teams

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics*
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers*
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks*
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
...,...,...,...,...,...,...,...,...,...
13,51,31,.622,—,116.9,113.0,3.60,2023,Memphis Grizzlies*
14,42,40,.512,9.0,114.4,112.5,1.63,2023,New Orleans Pelicans*
15,38,44,.463,13.0,114.2,114.1,-0.14,2023,Dallas Mavericks
16,22,60,.268,29.0,110.7,118.6,-7.62,2023,Houston Rockets


In [33]:
teams.to_csv("teams.csv")